In [1]:
import pandas as pd
from cyvcf2 import VCF

In [6]:
vcf = list(VCF('/Users/quinlan/Documents/Quinlan-PhD/LeeSixData/PD28690-chr21-snps-chunk.vcf'))


In [7]:
variant = vcf[123]
# *** variant.POS gives you vcf coordinate; variant.start and variant.end give you bed coordinates)
print(variant.CHROM, variant.POS, variant.start, variant.end, variant.REF, variant.ALT)
print(variant.INFO.get('variation_type'))
print(variant.format('filter')[0]) # this will either be PASS or a semi-colon separated list of filters this variant failed
print('alt_depth: ', variant.format('alt_depth')[0][0])
print('VAF: ', variant.format('AF')[0][0])
print('context: ', variant.format('context')[0])

21 10701128 10701127 10701128 T ['C']
None


KeyError: b'filter'

In [4]:
from collections import defaultdict

# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

# Initialize the dictionary to hold the count of each configuration
config_counts = defaultdict(int)

# Loop through each variant in the VCF
for v in vcf:
    # Get the genotypes for the samples
    genotypes = v.gt_types

    # Assign sample context


    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") >= 12:
        if blood == youngsperm == oldsperm:
            continue
        elif blood == UNKNOWN:
            continue
        elif youngsperm == oldsperm == UNKNOWN:
            continue
        elif blood == youngsperm and oldsperm == UNKNOWN:
            continue
        elif blood == oldsperm and youngsperm == UNKNOWN:
            continue
        else:
    # Create a tuple of the genotype configuration
            config = (youngsperm, blood, oldsperm)

    # Increment the count for this configuration
            config_counts[config] += 1


# Print the configuration counts
for config, count in config_counts.items():
    print(f"{config}: {count} occurrences")

(1, 1, 0): 57 occurrences
(1, 1, 3): 87 occurrences
(1, 0, 0): 12 occurrences
(1, 0, 3): 2 occurrences
(1, 0, 2): 6 occurrences
(3, 3, 0): 1 occurrences
(1, 3, 0): 5 occurrences
(1, 3, 2): 4 occurrences
(1, 3, 1): 12 occurrences
(0, 1, 2): 3 occurrences
(1, 3, 3): 14 occurrences
(1, 0, 1): 1 occurrences
(0, 1, 3): 4 occurrences
(3, 1, 2): 1 occurrences
(3, 1, 1): 3 occurrences
(3, 0, 2): 1 occurrences
(3, 1, 3): 8 occurrences
(3, 1, 0): 4 occurrences
(3, 0, 1): 1 occurrences
(0, 1, 0): 3 occurrences


In [5]:
def get_info_v(v, blood_col, sperm1_col, sperm2_col):
    genotypes = v.gt_types
    alt_depths = v.gt_alt_depths
    ref_depths = v.gt_ref_depths
    bases = v.gt_bases

    #assign sample context: GT
    youngsperm = genotypes[sperm1_col]
    blood = genotypes[blood_col]
    oldsperm = genotypes[sperm2_col]

    #assign sample context: AD
    ys_altdepth = alt_depths[sperm1_col]
    os_altdepth = alt_depths[sperm2_col]
    blood_altdepth = alt_depths[blood_col]

    #assign sample context: ref_depths
    ys_refdepth = ref_depths[sperm1_col]
    os_refdepth = ref_depths[sperm2_col]
    blood_refdepth = ref_depths[blood_col]

    #assign sample context: ref_depths
    ys_bases = bases[sperm1_col]
    os_bases = bases[sperm2_col]
    blood_bases = bases[blood_col]

    return youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases

def get_interesting(v, youngsperm, blood, oldsperm):
    if blood == youngsperm == oldsperm:
        return False
    elif blood == UNKNOWN:
        return False
    elif youngsperm == oldsperm == UNKNOWN:
        return False
    elif blood == youngsperm and oldsperm == UNKNOWN:
        return False
    elif blood == oldsperm and youngsperm == UNKNOWN:
        return False
    else:
        return True

In [7]:
# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

blood_col = 1
sperm1_col = 0
sperm2_col = 2

# variant.gt_types, variant.gt_ref_depths, variant.gt_alt_depths # numpy arrays
# variant.gt_phases, variant.gt_quals, variant.gt_bases # numpy array

for v in vcf:
    youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases = get_info_v(v,
                                                                                                                        blood_col, sperm1_col, sperm2_col)
    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10 and get_interesting(v, youngsperm, blood, oldsperm):
        print("blood", blood_bases, "alt depth", blood_altdepth, "ref depth", blood_refdepth, "young sperm", ys_bases, "alt depth", ys_altdepth,
                  "ref depth", ys_refdepth, "old sperm",os_bases, "alt depth", os_altdepth, "ref depth", os_refdepth)
        #print("blood", "sperm1", "sperm2")
        #print(v.INFO.get("AC"))
            #print(v.gt_types, ys_altdepth, os_altdepth, v.QUAL)
            #print(os_refdepth, ys_refdepth, blood_refdepth)

blood T/C alt depth 3 ref depth 1 young sperm T/C alt depth 3 ref depth 4 old sperm T/T alt depth 0 ref depth 3
blood T/G alt depth 1 ref depth 2 young sperm T/G alt depth 4 ref depth 3 old sperm G/G alt depth 2 ref depth 0
blood G/A alt depth 1 ref depth 2 young sperm G/A alt depth 4 ref depth 4 old sperm A/A alt depth 2 ref depth 0
blood A/T alt depth 2 ref depth 2 young sperm A/T alt depth 4 ref depth 5 old sperm A/A alt depth 0 ref depth 1
blood C/G alt depth 4 ref depth 5 young sperm C/G alt depth 5 ref depth 4 old sperm C/C alt depth 0 ref depth 1
blood C/A alt depth 3 ref depth 4 young sperm C/A alt depth 4 ref depth 5 old sperm A/A alt depth 2 ref depth 0
blood T/T alt depth 1 ref depth 2 young sperm T/A alt depth 6 ref depth 3 old sperm T/T alt depth 0 ref depth 2
blood G/G alt depth 1 ref depth 2 young sperm G/A alt depth 6 ref depth 3 old sperm A/A alt depth 1 ref depth 0
blood C/C alt depth 1 ref depth 1 young sperm C/G alt depth 4 ref depth 5 old sperm C/C alt depth 0 ref 

In [8]:
for v in vcf:
    # Get the necessary info
    genotypes = v.gt_types

    # Assign sample context: GT
    youngsperm = genotypes[0]
    blood = genotypes[1]
    oldsperm = genotypes[2]

    # Access FORMAT fields
    ro_blood = v.format('RO')[1]
    ao_blood = v.format('AO')[1]

    ro_youngsperm = v.format('RO')[0]
    ao_youngsperm = v.format('AO')[0]

    ro_oldsperm = v.format('RO')[2]
    ao_oldsperm = v.format('AO')[2]

    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10 and get_interesting(v, youngsperm, blood, oldsperm):

    # Print the FORMAT fields
        print(f"Blood - RO: {ro_blood}, AO: {ao_blood}")
        print(f"Young Sperm - RO: {ro_youngsperm}, AO: {ao_youngsperm}")
        print(f"Old Sperm - RO: {ro_oldsperm}, AO: {ao_oldsperm}")



Blood - RO: [1], AO: [3]
Young Sperm - RO: [4], AO: [3]
Old Sperm - RO: [3], AO: [0]
Blood - RO: [2], AO: [1]
Young Sperm - RO: [3], AO: [4]
Old Sperm - RO: [0], AO: [2]
Blood - RO: [2], AO: [1]
Young Sperm - RO: [4], AO: [4]
Old Sperm - RO: [0], AO: [2]
Blood - RO: [2], AO: [2]
Young Sperm - RO: [5], AO: [4]
Old Sperm - RO: [1], AO: [0]
Blood - RO: [5], AO: [4]
Young Sperm - RO: [4], AO: [5]
Old Sperm - RO: [1], AO: [0]
Blood - RO: [4], AO: [3]
Young Sperm - RO: [5], AO: [4]
Old Sperm - RO: [0], AO: [2]
Blood - RO: [2], AO: [1]
Young Sperm - RO: [3], AO: [6]
Old Sperm - RO: [2], AO: [0]
Blood - RO: [2], AO: [1]
Young Sperm - RO: [3], AO: [6]
Old Sperm - RO: [0], AO: [1]
Blood - RO: [1], AO: [1]
Young Sperm - RO: [5], AO: [4]
Old Sperm - RO: [1], AO: [0]
Blood - RO: [3], AO: [1]
Young Sperm - RO: [7], AO: [6]
Old Sperm - RO: [-2147483648], AO: [-2147483648]
Blood - RO: [4], AO: [0]
Young Sperm - RO: [6], AO: [4]
Old Sperm - RO: [1], AO: [0]
Blood - RO: [4], AO: [1]
Young Sperm - RO: [6

In [10]:
# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

blood_col = 1
sperm1_col = 0
sperm2_col = 2

# variant.gt_types, variant.gt_ref_depths, variant.gt_alt_depths # numpy arrays
# variant.gt_phases, variant.gt_quals, variant.gt_bases # numpy array

for v in vcf:
    youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases = get_info_v(v,
                                                                                                                        blood_col, sperm1_col, sperm2_col)
    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10 and get_interesting(v, youngsperm, blood, oldsperm):
        if youngsperm == blood == HET and oldsperm == HOM_ALT:
            print(v.CHROM, v.start, v.end,"blood", blood_bases, "alt depth", blood_altdepth, "ref depth", blood_refdepth, "young sperm", ys_bases, "alt depth", ys_altdepth,
                  "ref depth", ys_refdepth, "old sperm",os_bases, "alt depth", os_altdepth, "ref depth", os_refdepth)

chr21 5216972 5216973 blood T/G alt depth 1 ref depth 2 young sperm T/G alt depth 4 ref depth 3 old sperm G/G alt depth 2 ref depth 0
chr21 5217051 5217052 blood G/A alt depth 1 ref depth 2 young sperm G/A alt depth 4 ref depth 4 old sperm A/A alt depth 2 ref depth 0
chr21 5217606 5217607 blood C/A alt depth 3 ref depth 4 young sperm C/A alt depth 4 ref depth 5 old sperm A/A alt depth 2 ref depth 0
chr21 5218613 5218614 blood G/T alt depth 4 ref depth 1 young sperm G/T alt depth 5 ref depth 5 old sperm T/T alt depth 2 ref depth 0
chr21 5220199 5220200 blood A/G alt depth 2 ref depth 1 young sperm A/G alt depth 5 ref depth 3 old sperm G/G alt depth 1 ref depth 0
chr21 5220232 5220233 blood G/T alt depth 4 ref depth 1 young sperm G/T alt depth 5 ref depth 6 old sperm T/T alt depth 1 ref depth 0
chr21 5220327 5220331 blood ACCA/ACCG alt depth 4 ref depth 0 young sperm ACCA/ACCG alt depth 8 ref depth 0 old sperm ACCA/ACCA alt depth 1 ref depth 0
chr21 5220387 5220388 blood G/A alt depth 2 